### open csv files from directory and create markdown tables

- get csv files from the local build input/image/source folder
- print out markdown files
- save files as input/includes files
- !check out how I made excel files...

**IF EXCEL IS SOURCE USE CONVERT TO CSV**

In [1]:
from pathlib import Path
import pandas as pd
import csv
from json import loads
from IPython.display import Markdown, display

In [2]:
# get path from IG output dir
in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/images/data-element-mapping.csv')
# in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/includes/ballot-intro.csv')
out_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out')

in_path

PosixPath('/Users/ehaas/Documents/FHIR/davinci-ecdx/input/images/data-element-mapping.csv')

### load CSV using pandas

trim all strings too


In [3]:
df = pd.read_csv(in_path)
df.fillna('-', inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df

,No,Data Element,X12 277RFAI-v6020,X12n 278 Response-v5010,X12n 275-v6020,CDex $submit-attachment Parameter,CDex Request Attachment Task Profile Element,Submit Attachments Comments,Request Attachments Comments,ClaimResponse (from PAS Response Bundle),PAS Response Bundle
0,1.0,Tracking ID,"Loop: 2200D Segment: TRN02 Notes: TRN01 = ""1""","Loop: 2000E Segment: TRN02 Notes: TRN01 = ""1"",...",Loop: 2000A Segment: TRN02,TrackingId,Task.identifier,"<span class=""bg-success"" markdown=""1"">For *uns...","<span class=""bg-success"" markdown=""1"">Payer-as...","ClaimResponse.identifier, ClaimResponse.item.c...","ClaimResponse = Bundle.entry[0].resource, Comm..."
1,2.0,Use,Claim,Prior Auth,Claim,AttachTo,Task.reasonCode,"Choice of ""claim"" or ""preauthorization""","Choice of ""claim"" or ""preauthorization""",preauthorization,"Fixed to ""preauthorization"""
2,3.0,Payer ID,"Loop: 2100A Segment: NM109 Notes: NM108 = ""PI""","Loop: 2010A Segment: NM109 Notes: NM108 = ""PI""",Loop: 1000A Segment: NM109,PayerId,Task.requester.identifier,Payer ID,Payer ID,ClaimResponse.insurer: Organization.identifier,"ClaimResponse = Bundle.entry[0].resource, Orga..."
3,4.0,Payer URL,"Loop: 2100A Segment: PER04 Notes: PER03 = ""UR""","Loop: 2010A Segment: PER08 Notes: PER07 = ""UR""",-,(operation endpoint),"""payer-url"" Task.input",Payer endpoint where the attachments are submi...,Payer endpoint where the attachments are submi...,out of band,out of band
4,5.0,Organization ID,Loop: 2100B Segment: NM109 Notes: NM108 = id c...,Loop: 2010B Segment: NM109 Notes: NM101 = id c...,"Loop: 1000B Segment: NM109 Notes: NM108=""46""",OrganizationId,PractitionerRole.practitioner.identifier,Organization of provider who submitted claim/p...,Organization of provider who submitted claim/p...,ClaimResponse.requester: Organization.identifi...,"ClaimResponse = Bundle.entry[0].resource, Orga..."
5,6.0,Provider ID,Loop: 2100C Segment: NM109 Notes: NM108 = id c...,Loop: 2010EA Segment: NM109 Notes: NM101 = id ...,"Loop: 1000C Segment: NM109 Notes: NM108=""XX""",ProviderId,PractitionerRole.organization.identifier,Provider who submitted claim/prior authorization,Provider who submitted claim/prior authorization,ClaimResponse.requester: PractitionerRole.prac...,"ClaimResponse = Bundle.entry[0].resource, Prac..."
6,7.0,Claim/PreAuth ID,"Loop: 2200D Segment: REF02 Notes: REF01=""X1""","Loop: 2000F Segment: TRN02 Notes: TRN01 =""1"" o...","Loop: 1000D Segment: Ref02 Notes: Ref01=""X1"", ...",-,Task.reasonReference.identifier,"<span class=""bg-success"" markdown=""1"">Provider...","<span class=""bg-success"" markdown=""1"">Provider...",ClaimResponse.request: Claim.identifier,"ClaimResponse = Bundle.entry[0].resource, Clai..."
7,8.0,Line Item(s),"Loop: 2220D Notes: SVC01 = id code qualifier, ...","Loop: 2000F Segment: TRN02 Notes: TRN01 = ""1""",Loop: 2000A Segment: STC10 Notes: Used to conv...,Attachment.LineItem,“code” Task.input.extension,Claim/prior authorization line item numbers,Claim/prior authorization line item numbers,ClaimResponse.item.extension:itemTraceNumber ...,ClaimResponse = Bundle.entry[0].resource
8,9.0,LOINC Attachment Code,"Loop: 2220D Segments: STC01-02, STC10-02, STC1...",Loop: 2000E Segment: HI?? or Loop: 2000F Segme...,Loop: 2000A Segment: STC01-01,Attachment.Code,“code” Task.input,LOINC attachment codes,LOINC attachment codes,ClaimResponse.item.commmunicationRequest: Comm...,ClaimResponse = Bundle.entry[0].resource
9,10.0,Due Date,"Loop: 2200D Segment: DTP03 Notes: DTP01 = ""106...",-,-,-,Task.restriction.period,Deadline for submitting attachments to Payer,Deadline for submitting attachments to Payer,?,-


In [4]:
for i,col in enumerate(df.columns):
    print(i,col)

0 No
1 Data Element
2 X12 277RFAI-v6020
3 X12n 278 Response-v5010
4 X12n 275-v6020
5 CDex $submit-attachment Parameter
6 CDex Request Attachment Task Profile Element
7 Submit Attachments Comments
8 Request Attachments Comments
9 ClaimResponse (from PAS Response Bundle)
10 PAS Response Bundle


In [5]:
for i in df.index:
    print(i,df.loc[i,'Data Element'])

0 Tracking ID
1 Use
2 Payer ID
3 Payer URL
4 Organization ID
5 Provider ID
6 Claim/PreAuth ID
7 Line Item(s)
8 LOINC Attachment Code
9 Due Date
10 Date of Service
11 Member ID
12 Patient Name
13 Patient Account No.
14 Date of Birth
15 Questionnaire
16 QuestionnaireResponse
17 -
18 -
19 -
20 -
21 -
22 -


In [6]:
def editor_warning(my_file):
  return f'''<!-- {my_file}
*****************************************************************************************************
*                                  WARNING: DO NOT EDIT THIS FILE                                   *
*                                                                                                   *
* This file is generated by csv_to_markdown_tabler.ipynb. Any edits you make to this file will be   *
* overwritten                                                                                       *
* To change the contents of this file, edit input/images/{in_path.name}                     *
*****************************************************************************************************
-->\n\n'''

download_link = f'''The data element mapping table is available as a [CSV](data-element-mapping.csv) and [Excel](data-element-mapping.xlsx) file.'''

# convert to markdown table 
# select columns:

# maps = {#'data_elements_to_submit' : ([0,1,3,4,5,7,8,10,11],[1,4,5,6,8]),
#         'attachments_to_275' : ([0,1,3,4,5,7,8,10,11],[1,6,4,5,8]),
#         #'attachments_to_requests_277_278' : ([0,1,3,4,5,7,8,10,11],[5,6,2,3]),
#         'attachments_to_requests' : ([0,1,3,4,5,7,8,10,11],[1,6,7]),
#         #'data_elements_to_requests': (None,[1,6,7]),
        
#         'requests-277_278': (None,[1,7,2,3,9])
#         }
# this maps highlights new stuff in columns 2-4, using columns 9-11 as the source
maps = {
        'attachments_to_275' : ([0,1,2,3,4,5,7,8,10,11],[1,5,4,7]),
        'attachments_to_requests' : ([0,1,2,3,4,5,7,8,10,11],[1,5,6]),
        'q-attachments_to_requests' : ([0,1,2,3,4,5,7,8,10,11,15,16,],[1,5,6]),
        'requests-277_278': (range(0,15),[1,6,2,3,8]),
        'attachments_to_PAS_Bundle' : ([0,1,2,3,4,5,7,8,10,11],[1,9,10]),
        'PAS_Bundle_to_278' : ([0,1,2,3,4,5,7,8,10,11],[1,3,9,10]),
        'PAS_Bundle_to_parameters' : ([0,1,2,3,4,5,7,8,10,11],[1,5,9,10]),
         'PAS_Bundle_to_task' : ([0,1,2,3,4,5,7,8,10,11],[1,6,9,10]),
        }
# maps = {'text' : ([0,1,3,4,5,7,8,10,11],[1,6,5]) }
# maps = {'text' : (None,[0,1,2]) }

# df1 = df[['No','Data Element','CDex $submit-attachment Parameter', 'X12n 275']]
df1 = df.iloc[:, [0,1,5,4,7]]
for k,v in maps.items():
  if k != 'text':

    my_file =f'{k}.md'
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    with open(out_path / my_file, 'w') as md:
      my_table = df1.to_markdown(index=False, tablefmt="github")
      for i in range(1,10):
        my_table = my_table.replace('----','---')
        my_table = my_table.replace('  ',' ')
      print(f'======== my_file: {my_file} =======')
      display(Markdown(my_table))
      md.write(f'{editor_warning(my_file)}{my_table}\n{{:.grid}}\n\n{download_link}')
  else:
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    my_table = df1.to_markdown(index=False, tablefmt="github")
    for i in range(1,10):
      my_table = my_table.replace('----','---')
      my_table = my_table.replace('  ',' ')
    print(my_table)


======== my_file: attachments_to_275.md =======


| Data Element | CDex $submit-attachment Parameter | X12n 275-v6020 | Submit Attachments Comments |
|---|----|---------|-------------------|
| Tracking ID | TrackingId | Loop: 2000A Segment: TRN02 | <span class="bg-success" markdown="1">For *unsolicited* attachments, this is the provider-assigned tracking/control number. For *solicited* attachments, this is the payer-assigned tracking/control number.</span><!-- new-content --> |
| Use | AttachTo | Claim | Choice of "claim" or "preauthorization" |
| Payer ID | PayerId | Loop: 1000A Segment: NM109 | Payer ID |
| Payer URL | (operation endpoint) | - | Payer endpoint where the attachments are submitted using the $submit-attachment operation |
| Organization ID | OrganizationId | Loop: 1000B Segment: NM109 Notes: NM108="46" | Organization of provider who submitted claim/prior authorization |
| Provider ID | ProviderId | Loop: 1000C Segment: NM109 Notes: NM108="XX" | Provider who submitted claim/prior authorization |
| Line Item(s) | Attachment.LineItem | Loop: 2000A Segment: STC10 Notes: Used to convey status of the entire claim or a specific service line | Claim/prior authorization line item numbers |
| LOINC Attachment Code | Attachment.Code | Loop: 2000A Segment: STC01-01 | LOINC attachment codes |
| Date of Service | ServiceDate | Loop: 2100A Segment: DTP03 | Date of service for claim/prior authorization |
| Member ID | MemberId | Loop: 1000D Segment: NM109 Notes: NM108="MI" | Payer assigned patient identifier |

======== my_file: attachments_to_requests.md =======


| Data Element | CDex $submit-attachment Parameter | CDex Request Attachment Task Profile Element |
|---|----|-----|
| Tracking ID | TrackingId | Task.identifier |
| Use | AttachTo | Task.reasonCode |
| Payer ID | PayerId | Task.requester.identifier |
| Payer URL | (operation endpoint) | "payer-url" Task.input |
| Organization ID | OrganizationId | PractitionerRole.practitioner.identifier |
| Provider ID | ProviderId | PractitionerRole.organization.identifier |
| Line Item(s) | Attachment.LineItem | “code” Task.input.extension |
| LOINC Attachment Code | Attachment.Code | “code” Task.input |
| Date of Service | ServiceDate | “service-date” Task.input |
| Member ID | MemberId | Patient.identifier |

======== my_file: q-attachments_to_requests.md =======


...

| Data Element | X12n 278 Response-v5010 | ClaimResponse (from PAS Response Bundle) | PAS Response Bundle |
|---|---------------|------------|------------|
| Tracking ID | Loop: 2000E Segment: TRN02 Notes: TRN01 = "1", patient event tracking number. or Loop: 2000F Segment: TRN02 Notes: TRN01 = "1" or TRN01 = "2", service level tracking number | ClaimResponse.identifier, ClaimResponse.item.commmunicationRequest: CommunicationRequest.identifier | ClaimResponse = Bundle.entry[0].resource, CommunicationRequest = Bundle.entry[n].resource referenced by ClaimResponse.communincationRequest |
| Use | Prior Auth | preauthorization | Fixed to "preauthorization" |
| Payer ID | Loop: 2010A Segment: NM109 Notes: NM108 = "PI" | ClaimResponse.insurer: Organization.identifier | ClaimResponse = Bundle.entry[0].resource, Organization = Bundle.entry[n].resource referenced by ClaimResponse.insurer |
| Payer URL | Loop: 2010A Segment: PER08 Notes: PER07 = "UR" | out of band | out of band |
| Organization ID | Loop: 2010B Segment: NM109 Notes: NM101 = id code qualifier, NM102 = "2" | ClaimResponse.requester: Organization.identifier, ClaimResponse.requester: PractitionerRole.organization: Organziation.identifier | ClaimResponse = Bundle.entry[0].resource, Organization,PractitionerRole = Bundle.entry[n].resource referenced by ClaimResponse.requester |
| Provider ID | Loop: 2010EA Segment: NM109 Notes: NM101 = id code qualifier , NM102 = "1" | ClaimResponse.requester: PractitionerRole.practitioner: Practitioner.identifier | ClaimResponse = Bundle.entry[0].resource, PractitionerRole = Bundle.entry[n].resource referenced by ClaimResponse.requester |
| Line Item(s) | Loop: 2000F Segment: TRN02 Notes: TRN01 = "1" | ClaimResponse.item.extension:itemTraceNumber Note: CommunicationRequest.payload.extension:serviceLineNumber references this item | ClaimResponse = Bundle.entry[0].resource |
| LOINC Attachment Code | Loop: 2000E Segment: HI?? or Loop: 2000F Segment: PWK?? Notes: Clarify! | ClaimResponse.item.commmunicationRequest: CommunicationRequest.payload.extension:contentModifier | ClaimResponse = Bundle.entry[0].resource |
| Date of Service | Loop: 2000E (event level) or 2000F (line level) Segment: DTP03 Notes: DTP01="742" DTP02 = date format code | ClaimResponse.item.extension:requestedServiceDate | ClaimResponse = Bundle.entry[0].resource |
| Member ID | Loop: 2010C Segment: NM109 Notes: NM108 = id code qualifier | ClaimResponse.patient: Patient.identifer | ClaimResponse = Bundle.entry[0].resource, Patient = Bundle.entry[n].resource referenced by ClaimResponse.patient |

======== my_file: PAS_Bundle_to_parameters.md =======


| Data Element | CDex $submit-attachment Parameter | ClaimResponse (from PAS Response Bundle) | PAS Response Bundle |
|---|----|------------|------------|
| Tracking ID | TrackingId | ClaimResponse.identifier, ClaimResponse.item.commmunicationRequest: CommunicationRequest.identifier | ClaimResponse = Bundle.entry[0].resource, CommunicationRequest = Bundle.entry[n].resource referenced by ClaimResponse.communincationRequest |
| Use | AttachTo | preauthorization | Fixed to "preauthorization" |
| Payer ID | PayerId | ClaimResponse.insurer: Organization.identifier | ClaimResponse = Bundle.entry[0].resource, Organization = Bundle.entry[n].resource referenced by ClaimResponse.insurer |
| Payer URL | (operation endpoint) | out of band | out of band |
| Organization ID | OrganizationId | ClaimResponse.requester: Organization.identifier, ClaimResponse.requester: PractitionerRole.organization: Organziation.identifier | ClaimResponse = Bundle.entry[0].resource, Organization,PractitionerRole = Bundle.entry[n].resource referenced by ClaimResponse.requester |
| Provider ID | ProviderId | ClaimResponse.requester: PractitionerRole.practitioner: Practitioner.identifier | ClaimResponse = Bundle.entry[0].resource, PractitionerRole = Bundle.entry[n].resource referenced by ClaimResponse.requester |
| Line Item(s) | Attachment.LineItem | ClaimResponse.item.extension:itemTraceNumber Note: CommunicationRequest.payload.extension:serviceLineNumber references this item | ClaimResponse = Bundle.entry[0].resource |
| LOINC Attachment Code | Attachment.Code | ClaimResponse.item.commmunicationRequest: CommunicationRequest.payload.extension:contentModifier | ClaimResponse = Bundle.entry[0].resource |
| Date of Service | ServiceDate | ClaimResponse.item.extension:requestedServiceDate | ClaimResponse = Bundle.entry[0].resource |
| Member ID | MemberId | ClaimResponse.patient: Patient.identifer | ClaimResponse = Bundle.entry[0].resource, Patient = Bundle.entry[n].resource referenced by ClaimResponse.patient |

======== my_file: PAS_Bundle_to_task.md =======


| Data Element | CDex Request Attachment Task Profile Element | ClaimResponse (from PAS Response Bundle) | PAS Response Bundle |
|---|-----|------------|------------|
| Tracking ID | Task.identifier | ClaimResponse.identifier, ClaimResponse.item.commmunicationRequest: CommunicationRequest.identifier | ClaimResponse = Bundle.entry[0].resource, CommunicationRequest = Bundle.entry[n].resource referenced by ClaimResponse.communincationRequest |
| Use | Task.reasonCode | preauthorization | Fixed to "preauthorization" |
| Payer ID | Task.requester.identifier | ClaimResponse.insurer: Organization.identifier | ClaimResponse = Bundle.entry[0].resource, Organization = Bundle.entry[n].resource referenced by ClaimResponse.insurer |
| Payer URL | "payer-url" Task.input | out of band | out of band |
| Organization ID | PractitionerRole.practitioner.identifier | ClaimResponse.requester: Organization.identifier, ClaimResponse.requester: PractitionerRole.organization: Organziation.identifier | ClaimResponse = Bundle.entry[0].resource, Organization,PractitionerRole = Bundle.entry[n].resource referenced by ClaimResponse.requester |
| Provider ID | PractitionerRole.organization.identifier | ClaimResponse.requester: PractitionerRole.practitioner: Practitioner.identifier | ClaimResponse = Bundle.entry[0].resource, PractitionerRole = Bundle.entry[n].resource referenced by ClaimResponse.requester |
| Line Item(s) | “code” Task.input.extension | ClaimResponse.item.extension:itemTraceNumber Note: CommunicationRequest.payload.extension:serviceLineNumber references this item | ClaimResponse = Bundle.entry[0].resource |
| LOINC Attachment Code | “code” Task.input | ClaimResponse.item.commmunicationRequest: CommunicationRequest.payload.extension:contentModifier | ClaimResponse = Bundle.entry[0].resource |
| Date of Service | “service-date” Task.input | ClaimResponse.item.extension:requestedServiceDate | ClaimResponse = Bundle.entry[0].resource |
| Member ID | Patient.identifier | ClaimResponse.patient: Patient.identifer | ClaimResponse = Bundle.entry[0].resource, Patient = Bundle.entry[n].resource referenced by ClaimResponse.patient |